In [13]:
import numpy as np 
import scipy.stats as sps
import pandas as pd
import h5py
import os
from tqdm import tqdm

from copy import deepcopy

from numpy import array
from os import environ
from os.path import join
from sys import argv

from glob import glob
from numpy import zeros
from os.path import basename, join

from keras.models import load_model
from os import environ
from os.path import abspath, dirname, join

from copy import copy

from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import keras
from keras import backend as K

from keras.models import Sequential
from keras.models import model_from_json

from keras.layers import InputLayer, Input, Dense, Activation, Dropout, BatchNormalization, Reshape
from keras.layers import Average, Conv2D, MaxPooling2D, Flatten
from keras.layers.merge import Concatenate
from keras.activations import relu, softmax, sigmoid
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model

from keras.regularizers import l2

from keras.constraints import maxnorm

from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.losses import binary_crossentropy, mse

from keras.callbacks import LearningRateScheduler

In [ ]:
class TelegramCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        os.system("telegram-send 'Start NN training'")

    def on_train_end(self, logs={}):
        os.system("telegram-send 'End NN training'")

    def on_epoch_begin(self, epoch, logs={}):
        os.system("telegram-send 'Epoch {}'".format(epoch + 1))

    def on_epoch_end(self, epoch, logs={}):
        os.system("telegram-send 'loss: {:.4f}, val_loss: {:.4f}'".format(logs.get('loss'), logs.get('val_loss')))

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [115]:
class DataGenerator:
    def __init__(self, bad_columns=None, mean=None, sd=None):
        self.mean = mean
        self.sd = sd
        self.bad_columns = bad_columns
        
    def fit(self, X):
#         X = deepcopy(X).fillna(0)
        self.mean = X.mean()
        self.sd = X.std()
        print('end fit')
        
    def normalize(self, X):
        X  = (X - self.mean) / self.sd
        X = np.clip(X, a_min=-10, a_max=10)
        return X
    
    def do_fucking_job(self, X, y, feature_expander, 
                       features_win, forecast_win):
        
            assert min(features_win, forecast_win) > 0
            
            features_win -= 1
            forecast_win -= 1
            
            assert max(forecast_win, features_win) < X.shape[0]
            
            '''
                expander must return numpy array with shape (new_features_len)
            '''
            
            if feature_expander is not None:
                new_features = []
                for i in np.arange(features_win, X.shape[0]):
                    new_features.append(feature_expander(
                        X.iloc[i - features_win:i + 1]))
                new_features = np.array(new_features)
                wide = new_features.shape[1]
                dummy_cells = np.zeros((features_win, wide))
                new_features = np.concatenate((dummy_cells, new_features),
                                              axis=0)
                X = pd.concat((X, pd.DataFrame(new_features)),axis=1)
                
            if self.bad_columns is not None:
                X = X.drop(self.bad_columns, axis=1)
                
            Y = deepcopy(X)
            
            for i in np.arange(forecast_win) + 1:
                Y = pd.concat((Y, X.shift(periods=i)),axis=1)
            
            Y.columns = np.arange(len(Y.columns))
                
            Y = Y.fillna(Y.median(axis=0))
            return Y, y
            
    
    def flow(self, X, y, batch_size=10000, feature_expender=None, 
             forecast_win=10,features_win=100):
        ranges = np.arange(X.shape[0])
        assert batch_size <= len(ranges)
        while True:
            np.random.shuffle(ranges)
            for i in np.arange(0, len(ranges) - batch_size + 1, batch_size):
                inds = ranges[i:i + batch_size]
                x_batch, y_batch = X.iloc[inds], y[inds]
                x_batch, y_batch = self.do_fucking_job(x_batch, y_batch,
                                feature_expender, forecast_win, features_win)
                #  USE normalize if you REALLY need
                x_batch = self.normalize(x_batch)
                x_batch = np.array(x_batch)
                print('batch released')
                yield (x_batch, y_batch)     

In [103]:
a = np.arange(9).reshape(3,3)
b = np.arange(9).reshape(3,3)
c = np.concatenate((a,b),axis=1)
d = np.concatenate((a,b),axis=0)

In [9]:
c,d

(array([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5],
        [6, 7, 8, 6, 7, 8]]), array([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8],
        [0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]]))

In [116]:
df = pd.DataFrame({'B': [4, 5, 3, 2, 4], 'A': [4, 1, 3, 1, 0]})

In [117]:
df

,A,B
0,4,4
1,1,5
2,3,3
3,1,2
4,0,4


In [118]:
datagen = DataGenerator()

In [119]:
datagen.fit(df)

end fit


In [ ]:
l = [e for e in datagen.flow(df, np.array([0,1,0,1,1]), batch_size=2,
                             features_win=2, forecast_win=2)]

[0 1]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
0  4  4  4.0  4.0
1  1  5  4.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
2  3  3  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
2  3  3  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 4]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.Da

/Users/sergmiller/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5031: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  return_indexers=True)



<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
1  1  5  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[0 4]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
0  4  4  4.0  4.0
4  0  4  4.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
2  3  3  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[0 3]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0 

[4 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
0  4  4  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 3]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
3  1  2  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
2  3  3  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.Da

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[0 1]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
0  4  4  4.0  4.0
1  1  5  4.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
2  3  3  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[2 3]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
2  3  3  3.0  3.0
3  1  2  3.0  3.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch re

...

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
0  4  4  4.0  4.0
3  1  2  4.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
2  3  3  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[0 4]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
0  4  4  4.0  4.0
4  0  4  4.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  

0  4  4  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
0  4  4  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 3]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
3  1  2  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 1]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
1  1  5  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataF

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
2  3  3  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 4]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
4  0  4  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 3]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
3  1  2  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch re

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 1]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
1  1  5  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[2 4]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
2  3  3  3.0  3.0
4  0  4  3.0  3.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
0  4  4  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch re

<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
0  4  4  1.0  5.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[4 2]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
4  0  4  0.0  4.0
2  3  3  0.0  4.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[3 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
3  1  2  1.0  2.0
0  4  4  1.0  2.0 A    1.8
B    3.6
dtype: float64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
batch released
[1 0]    A  B
0  4  4
1  1  5
2  3  3
3  1  2
4  0  4
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
   0  1    2    3
1  1  5  1.0  5.0
0  4  4 

In [82]:
df - df.mean()

,A,B
0,2.2,0.4
1,-0.8,1.4
2,1.2,-0.6
3,-0.8,-1.6
4,-1.8,0.4


In [85]:
df.iloc[[1,3]] - df.mean()

,A,B
1,-0.8,1.4
3,-0.8,-1.6
